# Indicizzatore di Ricette con Elasticsearch

Questo script Python indicizza i file `.txt` delle ricette in **Elasticsearch** utilizzando l'**analyzer italiano**,  
consentendo la ricerca full-text sui titoli e contenuti delle ricette.

Automaticamente:
- Crea o resetta l'indice (`index_recipes`)
- Indicizza in bulk tutte le ricette dalla cartella `files/`
- Supporta query `match` e `match_phrase`


In [121]:
from elasticsearch import Elasticsearch, helpers
import time
import os

# Connessione a Elasticsearch
ES_HOST = "http://localhost:9200"
es = Elasticsearch([ES_HOST])

# Elimina l'indice se esiste già
if es.indices.exists(index='index_recipes'):
    es.indices.delete(index='index_recipes')

# Definizione del mapping per l'indice
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text",
                      "analyzer": "italian",
                      "search_analyzer": "italian"},
            "content": {"type": "text", 
                        "analyzer": "italian",
                        "search_analyzer": "italian"}
        }
    }
}

def bulk_index(directory, index_name):
    actions = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            # Rimuove l'estensione .txt per ottenere il titolo
            title = os.path.splitext(filename)[0]
            
            action = {
                "_index": index_name,
                "_source": {
                    "title": title,
                    "content": text
                }
            }
            actions.append(action)

    # Indicizza i documenti in bulk con una sola chiamata a Elasticsearch
    if actions:
        helpers.bulk(es, actions)
        print(f"{len(actions)} documenti indicizzati nell'indice '{index_name}'.")
    else:
        print("Nessun file .txt trovato nella directory.")

def search(query, query_text=""):
   # Aggiunge highlight alla query con tag personalizzati
   query['highlight'] = {
       "fields": {
           "title": {},
           "content": {"fragment_size": 200, "number_of_fragments": 1}
       },
       "pre_tags": ["<<<HIGHLIGHT>>>"],
       "post_tags": ["<<<ENDHIGHLIGHT>>>"]
   }
   
   res = es.search(index='index_recipes', body=query)
   total = res['hits']['total']['value']
   print(f"\n{'='*80}")
   if query_text:
       print(f"Risultati della ricerca per la query \'{query_text}\': {total} ricette trovate")
   else:
       print(f"Risultati della ricerca: {total} ricette trovate")
   print(f"{'='*80}\n")
   
   # Codici colore ANSI
   YELLOW = '\033[93m'
   RESET = '\033[0m'
   
   for i, hit in enumerate(res['hits']['hits'], 1):  # Mostra tutti i risultati
    res_doc = hit['_source']
    
    # Usa highlight se disponibile, altrimenti usa il testo normale
    if 'highlight' in hit:
        title = hit['highlight'].get('title', [res_doc['title']])[0]
        if 'content' in hit['highlight']:
            content_preview = hit['highlight']['content'][0] + "..."
        else:
            content_preview = res_doc['content'][:200] + "..." if len(res_doc['content']) > 200 else res_doc['content']
    else:
        title = res_doc['title']
        content_preview = res_doc['content'][:200] + "..." if len(res_doc['content']) > 200 else res_doc['content']
    
    # Sostituisce i tag di highlight con colori
    title = title.replace("<<<HIGHLIGHT>>>", YELLOW).replace("<<<ENDHIGHLIGHT>>>", RESET)
    content_preview = content_preview.replace("<<<HIGHLIGHT>>>", YELLOW).replace("<<<ENDHIGHLIGHT>>>", RESET)
    
    print(f"{i}. Titolo: {title}")
    print(f"   Contenuto: {content_preview}\n")

def parse_and_search(query):
    """
    Analizza la query utente ed esegue la ricerca Elasticsearch appropriata.
    
    Args:
        query (str): Input utente nel formato 'campo:termine' o 'campo:"frase esatta"'
                     oppure solo 'termine' per multi_match su entrambi i campi
                     Campi supportati: 'title', 'content'
    
    Returns:
        None: Stampa i risultati direttamente o messaggi di errore
    """
    # Controlla se è specificato un campo
    if ":" not in query:
        # Nessun campo specificato, usa multi_match su title e content
        content_query = query.strip()
        if content_query.startswith('"') and content_query.endswith('"'):
            phrase = content_query.strip('"')
            body = {
                "query": {
                    "multi_match": {
                        "query": phrase,
                        "fields": ["title", "content"],
                        "type": "phrase"
                    }
                },
                "size": 10000
            }
        else:
            body = {
                "query": {
                    "multi_match": {
                        "query": content_query,
                        "fields": ["title", "content"]
                    }
                },
                "size": 10000
            }
        search(body, query)
        return
    
    type_query, content_query = query.split(":", 1)
    type_query = type_query.strip().lower()
    content_query = content_query.strip()

    # Costruisce il body della query per il campo specifico
    if type_query in ["title", "content"]:
        if content_query.startswith('"') and content_query.endswith('"'):
            phrase = content_query.strip('"')
            body = {"query": {"match_phrase": {type_query: phrase}}, "size": 10000}
        else:
            body = {"query": {"match": {type_query: content_query}}, "size": 10000}
        search(body, query)
    else:
        print("Campo sconosciuto. Usa 'title:' o 'content:'.")


## Misurazione del Tempo di Indicizzazione

Questo snippet misura quanto tempo impiega il processo di indicizzazione.


In [122]:
# Indicizza i documenti dalla directory 'files'
before = time.time()
es.indices.create(index='index_recipes', body=mapping)
bulk_index("files", "index_recipes")
after = time.time()
print(f"L'indicizzazione ha richiesto {after - before} secondi")


5939 documenti indicizzati nell'indice 'index_recipes'.
L'indicizzazione ha richiesto 1.6523878574371338 secondi


## Query di Ricerca Interattiva (con risultati illimitati)

Questo snippet permette all'utente di inserire una query di ricerca in formato naturale e costruisce  
dinamicamente una query Elasticsearch, supportando:
- `multi_match` (cerca in titolo e contenuto)
- `match` (ricerca su campo singolo)
- `match_phrase` (ricerca frase esatta)


In [123]:
# Chiede all'utente di inserire una query
print("\nInserisci la tua query di ricerca usando uno dei seguenti formati:")
print('  • tiramisu (cerca in titolo e contenuto)')
print('  • title: tiramisu (cerca solo nel titolo)')
print('  • content: "burro e salvia" (frase esatta nel contenuto)')
print('  • content: banane (cerca solo nel contenuto)')
print("Usa le virgolette per cercare una frase esatta.\n")

query = input("Ricerca → ").strip()

# Usa la funzione refactorizzata
parse_and_search(query)



Inserisci la tua query di ricerca usando uno dei seguenti formati:
  • tiramisu (cerca in titolo e contenuto)
  • title: tiramisu (cerca solo nel titolo)
  • content: "burro e salvia" (frase esatta nel contenuto)
  • content: banane (cerca solo nel contenuto)
Usa le virgolette per cercare una frase esatta.


Risultati della ricerca per la query 'anice': 27 ricette trovate

1. Titolo: Moretta fanese
   Contenuto: Per preparare la moretta fanese, iniziate con la spuma all'anice: mescolate in una piccola caraffa il liquore all’anice con il brandy Unite il mix di alcolici alla panna liquida fresca in una ciotola montate...

2. Titolo: Pere al vino rosso
   Contenuto: il vino rosso lo zucchero e l'acqua in una casseruola (2-3) e portate lentamente a bollore.Eliminate la buccia alle pere senza togliere il picciolo tenete da parte.Unite nella casseruola la cannella l’anice...

3. Titolo: Pere cotte
   Contenuto: preparare le pere cotte iniziate pelandole intere con un pelapatate avendo cura 

## 10 query di test per l'analyzer italiano

Queste query testano **specificamente** i componenti della pipeline dell'analyzer italiano:

### Funzionalità dell'Analyzer (Test 1-8)
1. **lowercase filter** → case-insensitive (MOZZARELLA = mozzarella)
2. **italian_elision** → rimozione elisioni (l'origano → origano)
3. **stopwords removal** → "della" viene rimosso, matcha "panna"
4. **plurali** → mirtillo/mirtilli matchano 
5. **genere** → fritto/fritta/fritti/fritte matchano (variazioni morfologiche)
6. **light stemming** → pomodoro ≠ pomodorino (mantiene distinzioni semantiche)
7. **accenti** → tiramisu matcha Tiramisù
8. **match_phrase + elisione** → "spaghetti all'amatriciana" (frase esatta con elisione)

### Limitazioni (Test 9-10)
9. **false positives** → ciliegina = ciliegino (stemming troppo aggressivo in rari casi)
10. **derivazioni non gestite** → natalizi ≠ natale (stem diversi: nataliz vs natal)

In [124]:
# Test 1: Lowercase filter - test case-insensitive
query = 'MOZZARELLA'
parse_and_search(query)


Risultati della ricerca per la query 'MOZZARELLA': 150 ricette trovate

1. Titolo: Mozzarella ripiena
   Contenuto: Per realizzare le mozzarelle ripiene per prima cosa ritagliate la calotta di ciascuna mozzarella così da ottenere un incavo riducete a cubetti le calotte e teneteli da parte Ponete le mozzarelle scavate...

2. Titolo: Mozzarella fritta
   Contenuto: Ora potete servire i vostri bocconcini di mozzarella fritta ben caldi...

3. Titolo: Rotolo di mozzarella farcito
   Contenuto: Per realizzare il rotolo di mozzarella farcito, iniziate ponendo la mozzarella in una ciotola (scieglietene una poco più grande della mozzarella) con la propria acqua di conservazione e aggiungete altra...

4. Titolo: Spaghetti quadrati con sugo fresco e mozzarella
   Contenuto: la mozzarella...

5. Titolo: Mozzarella ripiena con cime di rapa
   Contenuto: DOP creando un buco a cono Estraete il pezzo di mozzarella Riempite la mozzarella DOP con le cime di rapa Trasferite in un piatto da portata Aggiu

In [134]:
# Test 2: Elision filter - "l'origano" diventa "origano"
query = "content: l'origano"
parse_and_search(query)


Risultati della ricerca per la query 'content: l'origano': 163 ricette trovate

1. Titolo: Caprese sfiziosa con salsa all'origano
   Contenuto: Per preparare la caprese sfiziosa con salsa all’origano cominciate preparando la salsa all’origano....

2. Titolo: Focaccia con pomodorini e origano
   Contenuto: La vostra focaccia con pomodorini e origano è pronta per essere gustata...

3. Titolo: Piadina con pomodori secchi e mozzarella di bufala
   Contenuto: Procedete allo stesso modo per preparare l'altra e gustate la vostra piadina con pomodori secchi, bufala e origano...

4. Titolo: Roselline di melanzane
   Contenuto: melanzane, poi eliminate le due estremità e tagliatele per il lato corto a fettine sottili di circa 5 millimetri di spessore il sale Trascorso questo tempo, sfornate le melanzane Aromatizzate con l’origano...

5. Titolo: Filetto di platessa alla sorrentina
   Contenuto: Tritate l’origano versate la farina distribuendola su un vassoio aggiungete l’origano tritato il sale 

In [126]:
# Test 3: Stopwords filter - "della" viene rimossa, quindi "panna della fattoria" matcha con "panna"
query = '"della panna"'
parse_and_search(query)


Risultati della ricerca per la query '"della panna"': 850 ricette trovate

1. Titolo: Gelato alla panna
   Contenuto: Il vostro gelato alla panna è pronto per essere gustato...

2. Titolo: Panna cotta
   Contenuto: punta di un coltello Mettete in un pentolino la panna e versate lo zucchero Aromatizzate con i semi Scaldate a fuoco basso il tutto e quando la panna sfiorerà il bollore, spegnete il fuoco e rimuovete...

3. Titolo: Arrosto alla panna
   Contenuto: Versate quindi la panna allungate con un po' di brodo e lasciate cuocere l’arrosto chiudendo il tegame con un coperchio Lasciate cuocere per poco più di un'ora (per capire se il vostro arrosto è cotto...

4. Titolo: Panna cotta al cioccolato
   Contenuto: Per preparare la panna cotta al cioccolato iniziate dalla panna cotta: mettete in ammollo i fogli di gelatina in una terrina con acqua fredda per almeno 10 minuti....

5. Titolo: Caffè goloso con panna
   Contenuto: Spremete la panna sul caffè Dovrete creare uno bel ciuffo che c

In [137]:
# Test 4: Light stemming - "mirtillo" vs "mirtilli" (devono matchare)
query = 'content:mirtillo'
parse_and_search(query)


Risultati della ricerca per la query 'content:mirtillo': 60 ricette trovate

1. Titolo: Torta ai mirtilli
   Contenuto: e servire la vostra torta ai mirtilli....

2. Titolo: Confettura di ribes rosso e mirtilli
   Contenuto: La vostra confettura di ribes rosso e mirtilli è pronta...

3. Titolo: Muffin carote e mirtilli
   Contenuto: su ogni muffin (in totale vi serviranno circa 35 g di mirtilli) lasciate intiepidire i vostri muffin di carote e mirtilli prima di servirli!...

4. Titolo: Biscotti ai mirtilli
   Contenuto: Sfornate i biscotti ai mirtilli, fateli intiepidire su una gratella e poi serviteli...

5. Titolo: Flan di Bettelmatt
   Contenuto: qualche mirtillo fresco...

6. Titolo: Aspic ai frutti di bosco
   Contenuto: Per realizzare l’aspic ai frutti di bosco iniziate lavando e asciugando i mirtilli A parte in una ciotola e ponete i fogli di gelatina in acqua fredda per 10 minuti Scolate bene con un colino i fogli di...

7. Titolo: Gingerbread bundt cake
   Contenuto: In una c

In [138]:
# Test 5: Genere - "fritto/fritta/fritti/fritte" (devono matchare)
query = 'content:fritto'
parse_and_search(query)


Risultati della ricerca per la query 'content:fritto': 148 ricette trovate

1. Titolo: Gallinella fritta
   Contenuto: Per preparare la gallinella fritta cominciate proprio dalla pulizia dei filetti del pesce....

2. Titolo: Piadina con sarde fritte e burrata
   Contenuto: qualche pomodorino giallo Chiudete la piadina con sarde fritte e burrata...

3. Titolo: Frutta fritta caramellata
   Contenuto: nell’acqua ghiacciata, che avrete predisposto in una ciotola, per far solidificare il caramello Sistemate la frutta fritta caramellata su un piatto da portata e servite...

4. Titolo: Costolette d'agnello fritte
   Contenuto: Per realizzare le costolette d’agnello fritte iniziate ritagliando dal carrè 8 costolette con un coltello insaporite con il Parmigiano Reggiano grattugiato salate rotolatele poi nel pangrattato nuovamente...

5. Titolo: Fagiolini fritti
   Contenuto: Per preparare i fagiolini fritti cominciate dalla loro pulizia....

6. Titolo: Ricotta fritta
   Contenuto: Per preparar

In [142]:
# Test 6a: Light stemming 1 - "pomodoro" vs "pomodorino" (NON devono matchare)
query = 'title:pomodorino'
parse_and_search(query)


Risultati della ricerca per la query 'title:pomodorino': 84 ricette trovate

1. Titolo: Pomodorini confit
   Contenuto: Per preparare i pomodorini confit, iniziate lavando i pomodorini sotto acqua corrente Asciugateli con un canovaccio o carta da cucina Ora disponete i pomodorini tagliati su una leccarda ricoperta di c...

2. Titolo: Pomodorini all'australiana
   Contenuto: Lavate ed asciugate i pomodorini, tagliateli in quarti e disponeteli , non troppo a ridosso l’uno dell’altro, su di una o più teglie foderate con carta forno. Cospargeteli con del sale fino (non tropp...

3. Titolo: Pomodorini in crosta
   Contenuto: Per preparare i pomodorini in crosta come prima cosa lavateli sotto l'acqua corrente Trasferiteli in una ciotola e conditeli con sale pepe e origano Mescolate Da ciascuna metà ricavate 8 striscioline ...

4. Titolo: Conserva di pomodorini
   Contenuto: Per preparare la conserva di pomodorini, dovrete sanificare i barattoli come indicato nel box in fondo alla ricetta, s

In [143]:
# Test 6b: Light stemming 1 - "pomodoro" vs "pomodorino" (NON devono matchare)
query = 'title:pomodoro'
parse_and_search(query)


Risultati della ricerca per la query 'title:pomodoro': 95 ricette trovate

1. Titolo: Minestra di pomodoro
   Contenuto: Per preparare la minestra di pomodoro come prima cosa lavate i pomodori. Poi tagliateli prima in 4 Tenete da parte e passate al cipollotto di Tropea; eliminate le foglie più esterne e tagliatelo a fet...

2. Titolo: Gelo di pomodoro
   Contenuto: Per preparare il gelo di pomodoro cominciate mettendo in ammollo la gelatina in acqua fresca Intanto passate ai pomodori: dopo averli staccati dai rametti, lavateli e asciugateli. Poi incideteli forma...

3. Titolo: Piccatine al pomodoro
   Contenuto: Per realizzare le piccatine al pomodoro per prima cosa tagliate a fette la fesa di vitello in modo da ricavare 4 pezzi Avvolgete ciascuna fetta nella carta forno quindi battetela con il batticarne Ott...

4. Titolo: Pomodori ripieni
   Contenuto: Per preparare i pomodori ripieni per prima cosa lavate i pomodori, tagliate la calotta e tenetela da parte Con l’aiuto di un coltell

In [ ]:
# Test 7: Ricerca matcha anche con accenti (Tiramisù)
query = 'tiramisu'
parse_and_search(query)


Risultati della ricerca per la query 'title: "tiramisu"': 34 ricette trovate

1. Titolo: Tiramisù
   Contenuto: Per preparare il tiramisù preparate il caffé con la moka per ottenerne 300 g, poi zuccherate a piacere (noi abbiamo messo un cucchiaino) e lasciatelo raffreddare in una ciotolina bassa e ampia. Separa...

2. Titolo: Torta tiramisù
   Contenuto: Per preparare la torta tiramisù iniziate dalla base di savoiardi; dovrete cuocere una base alla volta. Prendete le 4 uova necessarie per le basi e dividete i tuorli dagli albumi: le uova devono essere...

3. Titolo: Tiramisù alle fragole
   Contenuto: Per realizzare il tiramisù alle fragole lavate le fragole e poi togliete il picciolo quindi tagliatele a rondelle Unite anche 20 g di succo di limone Mescolate Unite 125 g di zucchero Una volta che lo...

4. Titolo: Tiramisù al pistacchio
   Contenuto: Per preparare il tiramisù al pistacchio ponete i pistacchi in un mixer reparate il caffè con la moka e lasciatelo raffreddare.Ponete sul 

In [146]:
# Test 7: Ricerca nel campo title - cerca solo nel titolo, non nel contenuto
query = 'title: "spaghetti alla carbonara"'
parse_and_search(query)


Risultati della ricerca per la query 'title: "carbonara di spaghetti"': 0 ricette trovate



In [ ]:
# Test 8: Match_phrase - frase esatta con elisione (all'amatriciana)
query = 'title: "spaghetti all\'amatriciana"'
parse_and_search(query)


Risultati della ricerca per la query 'title: "spaghetti all'amatriciana"': 1 ricette trovate

1. Titolo: Spaghetti all'Amatriciana
   Contenuto: Per preparare gli spaghetti all’amatriciana, per prima cosa mettete a bollire l'acqua per la cottura della pasta da salare poi a bollore. Potete quindi dedicarvi al condimento: prendete il guanciale, ...



In [132]:
# Test 9: Light stemming 2
# NOTA: Questo può generare false positive, ad esempio cercando "ciliegina" (del cocktail)
# vengono trovati anche i "pomodorini ciliegini" perché condividono lo stesso stem
query = 'ciliegina'
parse_and_search(query)


Risultati della ricerca per la query 'ciliegina': 25 ricette trovate

1. Titolo: Pasta con stracchino, bresaola e ciliegini
   Contenuto: Per realizzare la pasta con stracchino, bresaola e ciliegini per prima cosa ponete sul fuoco una pentola colma di acqua, portatela al bollore e a quel punto salate....

2. Titolo: Penne alla crudaiola
   Contenuto: Per preparare le penne alla crudaiola lavate e tagliate a quarti i pomodorini ciliegino Spuntate, lavate e tagliate a rondelline sottili le zucchine novelle In una ciotola mettete i pomodorini, i funghi...

3. Titolo: Noodles di zucchine e carote con tonno e pomodoro
   Contenuto: striscioline Sovrapponete due strisce per volta, ripiegatele su stesse e poi ritagliate delle listarelle sottili Trasferite le carote e le zucchine all’interno di un ampia ciotola Lavate i pomodorini ciliegino...

4. Titolo: Verza alla salentina
   Contenuto: UniTe i pomodorini ciliegini tagliati a metà la verza scolata salate e coprite con acqua Cuocete per 30 

In [133]:
# Test 10: Limitazioni dello stemming - "natalizi" NON matcha con "natale"
# Aspetto negativo: lo stemming non sempre unisce parole semanticamente correlate
# (natalizio → nataliz, natale → natal, stem diversi!)
query = 'natalizi'
parse_and_search(query)


Risultati della ricerca per la query 'natalizi': 9 ricette trovate

1. Titolo: Pacchetti natalizi con scampi, zenzero e lime
   Contenuto: Per realizzare i pacchetti natalizi con scampi, zenzero e lime per prima cosa occupatevi della pulizia degli scampi: staccate la testa con delle forbici incidete il carapace Estraete delicatamente il filamento...

2. Titolo: Cestini di pasta kataifi con Nutella®
   Contenuto: circa 25 minuti Lasciate raffreddare i cestini e poi, delicatamente, staccateli dagli stampini Versate Nutella® in una sac-à-poche con bocchetta liscia di 2-3 mm riempiendo i cestini Date un tocco natalizio...

3. Titolo: Pavesini candy cane
   Contenuto: Decorate in ultimo con gli zuccherini natalizi Tenete da parte i biscottini decorati. Trasferire la ganache rassodata ma malleabile in un sac-à-poche con bocchetta liscia da 1 cm....

4. Titolo: Calendario dell'avvento di biscotti
   Contenuto: Impastate brevemente e formate un panetto, appiattitelo prima di avvolgerlo nella 